<a href="https://colab.research.google.com/github/dantae74/machine-learning/blob/main/stock-etc/01-crawling-naver-sise-market-sum-500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 네이버에서 시가총액 상위 500개 crawling

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

In [ ]:
def crawl(url):
  data = requests.get(url)
  # print(data)
  return data.content

In [ ]:
def getSiseMarketSum(sosok, page):
  url = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok={0}&page={1}".format(sosok, page)
  pageString = crawl(url)
  list = parse(pageString)
  return list

In [ ]:
def parse(pageString):
  bsObj = BeautifulSoup(pageString, "html.parser")
  # print(bsObj)
  box_type_l = bsObj.find("div", {"class":"box_type_l"})
  # print(box_type_l)

  type_2 = box_type_l.find("table", {"class":"type_2"})
  # print(type_2)

  tbody = type_2.find("tbody")
  # print(tbody)

  trs = tbody.findAll("tr")
  # print(len(trs))

  stockinfos = []
  for tr in trs:
    try:
      stockInfo = getStockInfo(tr)
      # print("\nstockInfo:\n",stockInfo)
      stockinfos.append(stockInfo)
    except Exception as e:
        # print(e)
        pass
  
  return stockinfos


In [ ]:
def getStockInfo(tr):
  tds = tr.findAll("td")
  # print(tds)

  # 순위
  rank = tds[0].text
  # print("\ntd:", tds[0])

  # a tag => href
  aTag = tds[1].find("a")
  href = aTag["href"]
  
  # name
  name = tds[1].text

  # 현재가
  nowPrice = tds[2].text

  # 전일대비 가격

  # 시가총액
  totalPrice = tds[6].text

  # 거래량
  volume = tds[9].text

  return {"rank":rank, "href":href, "code":href[20:], 
          "name":name, "nowPrice":nowPrice, "totalPrice":totalPrice,
          "volume":volume}

In [ ]:
# list = getSiseMarketSum(1, 2)

In [ ]:
domestic_stocks = {"kospi":0, "kosdaq":1}

for key, value in domestic_stocks.items():
  result = []
  for i in range(1, 10 + 1): # 500 = 50 * 10
    list = getSiseMarketSum(int(value), i)
    result += list

  print(result)
  print(len(result))

  # jshon 형식으로 파일에 저장
  # file_path = 'https://github.com/dantae74/machine-learning/tree/main/dataset/{}.json'.format(key)
  # file = open(file_path, "w+")

  file = open("./{}.json".format(key), "w+")
  file.write(json.dumps(result))
